# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [9]:
from pathlib import Path
from pprint import pprint

pprint(sorted(Path().resolve().glob("*")))

file_path_list = sorted(Path().resolve().glob("event_data/**/*"))
pprint(file_path_list)

[PosixPath('/workspace/home/.ipynb_checkpoints'),
 PosixPath('/workspace/home/.workspace-config.json'),
 PosixPath('/workspace/home/Project_1B_ Project_Template.ipynb'),
 PosixPath('/workspace/home/event_data'),
 PosixPath('/workspace/home/images')]
[PosixPath('/workspace/home/event_data/2018-11-01-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-02-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-03-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-04-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-05-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-06-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-07-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-08-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-09-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-10-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11-11-events.csv'),
 PosixPath('/workspace/home/event_data/2018-11

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [11]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


In [3]:
import pandas as pd

path = "event_datafile_new.csv"
df=pd.read_csv(path)
df

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Des'ree,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",139,You Gotta Be,8
1,Mr Oizo,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",139,Flat 55,8
2,Tamba Trio,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",139,Quem Quiser Encontrar O Amor,8
3,The Mars Volta,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",139,Eriatarka,8
4,Infected Mushroom,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",139,Becoming Insane,8
5,Blue October / Imogen Heap,Kaylee,F,7,Summers,241.39710,free,"Phoenix-Mesa-Scottsdale, AZ",139,Congratulations,8
6,Girl Talk,Kaylee,F,8,Summers,160.15628,free,"Phoenix-Mesa-Scottsdale, AZ",139,Once again,8
7,Black Eyed Peas,Sylvie,F,0,Cruz,214.93506,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",9,Pump It,10
8,Fall Out Boy,Ryan,M,1,Smith,200.72444,free,"San Jose-Sunnyvale-Santa Clara, CA",169,Nobody Puts Baby In The Corner,26
9,M.I.A.,Ryan,M,2,Smith,233.71710,free,"San Jose-Sunnyvale-Santa Clara, CA",169,Mango Pickle Down River (With The Wilcannia Mob),26


In [16]:
print(df.columns.tolist())

['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']


### EDA = Exploratory Data Analysis

In [13]:
# df[["sessionId","userId"]].drop_duplicates().sort_values(by="sessionId")

# IMPORTANT: sessionID is unique globally. In other words, one sessionID maps to only one userID.
(df.groupby('sessionId')['userId'].nunique() != 1).sum()

0

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [19]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [20]:
# TO-DO: Create a Keyspace 
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [21]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [1]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# necessary WHERE: sessionID, itemInSession, userID, songname

# is sessionid specific per user or unique?

# sessionId, itemInSession - artist, song, length
# sessionId, userId - artist, song, firstName, lastName, sort by itemInSession)
# song - firstName, lastName

# --> Decision:
# Similar to https://tech.ebayinc.com/engineering/cassandra-data-modeling-best-practices-part-1/
# 1. One table by (sessionId, ItemInSession), userID is really unnecessary, but maybe include as clustering col?
# 2. Second table by (song, userId) - firstname, lastname  # could be duplicates along session!
                    

# Create table

In [101]:
# Create tables
# ['artist', 'firstName', 'gender', 'itemInSession', 'lastName', 'length', 'level', 'location', 'sessionId', 'song', 'userId']


session.execute("DROP TABLE by_session;")
session.execute("DROP TABLE by_song;")

query = """
CREATE TABLE IF NOT EXISTS by_session (
  sessionId int,
  itemInSession int,
  artist text,
  song text,
  length float,
  userId int,
  firstName text,
  lastName text, 
  PRIMARY KEY (sessionId, itemInSession)
);
""".strip()
session.execute(query)

query = """
CREATE TABLE IF NOT EXISTS by_song (
    song text,
    userId int,
    firstName text,
    lastName text, 
    PRIMARY KEY (song, userId)
);
""".strip()
session.execute(query)

In [102]:
# cassandra seems to store all names in lower case letters...
x=session.execute("SELECT column_name, type FROM system_schema.columns WHERE keyspace_name = 'udacity' AND table_name = 'by_session';")
for row in x:
    print(row)


Row(column_name='artist', type='text')
Row(column_name='firstname', type='text')
Row(column_name='iteminsession', type='int')
Row(column_name='lastname', type='text')
Row(column_name='length', type='float')
Row(column_name='sessionid', type='int')
Row(column_name='song', type='text')
Row(column_name='userid', type='int')


In [43]:
dct = df.iloc[0,:].to_dict()
dct

{'artist': "Des'ree",
 'firstName': 'Kaylee',
 'gender': 'F',
 'itemInSession': 1,
 'lastName': 'Summers',
 'length': 246.30812000000003,
 'level': 'free',
 'location': 'Phoenix-Mesa-Scottsdale, AZ',
 'sessionId': 139,
 'song': 'You Gotta Be',
 'userId': 8}

In [110]:
# helper function to construct insert commands

def _to_str(x):
    if isinstance(x, str):
        x = x.replace("'", "") # Des'ree -> Desree. Otherwise we get syntax error elow
        return f"'{x}'"  # otherwise 'You gotta be' is not parsed as one string
    else:
        return str(x)  # otherwise float/int values throw an error

def _get_query(dct, prefix, keys, suffix=""):
    # note: keys need to be in order of table definition, but needs to be taken care of by user
    d = {k: dct[k] for k in keys}
    joined_keys = ", ".join(d.keys())
    joined_vals = ", ".join([_to_str(x) for x in d.values()])
    return f"{prefix} ({joined_keys}) VALUES ({joined_vals}){suffix};"


def insert1(dct):
    return _get_query(dct, "INSERT INTO by_session", ["sessionId", "itemInSession", "artist", "song", "length", "userId", "firstName", "lastName"])

def insert2(dct):
    return _get_query(dct, "INSERT INTO by_song", ["song", "userId", "firstName", "lastName"], " IF NOT EXISTS")

insert1(dct)
insert2(dct)

"INSERT INTO by_song (song, userId, firstName, lastName) VALUES ('Anything You Say (Unreleased Version)', 5, 'Elijah', 'Davis') IF NOT EXISTS;"

In [104]:
# insert stuff
from tqdm import tqdm

def _exec(dct, func):
    try:
        query = func(dct)
        session.execute(query)
    except Exception as err:
        print(dct)
        print(query)
        print(err)
        raise err

for idx, row in tqdm(df.iterrows(), total=len(df)):
    dct = row.to_dict()
    
    _exec(dct, insert1)
    _exec(dct, insert2)


100%|██████████| 6820/6820 [00:44<00:00, 153.92it/s]


# Queries

In [105]:
### 1. Give me the artist, song title and song's length in the music app history that was heard
# during  sessionId = 338, and itemInSession  = 4

def _query(q):
    for row in session.execute(q.strip()):
        print(row)

q="""
SELECT artist,song,length
FROM by_session
WHERE sessionID=338 AND itemInSession=4
"""
_query(q)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [106]:
### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)
# for userid = 10, sessionid = 182

q="""
SELECT artist,song,firstname, lastname, userid, iteminsession
FROM by_session
WHERE sessionID=182
"""
_query(q)

Row(artist='Down To The Bone', song='Keep On Keepin On', firstname='Sylvie', lastname='Cruz', userid=10, iteminsession=0)
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz', userid=10, iteminsession=1)
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz', userid=10, iteminsession=2)
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz', userid=10, iteminsession=3)


In [107]:
### 3. Give me every user name (first and last) in my music app history
# who listened to the song 'All Hands Against His Own'

q="""
SELECT userId, firstname, lastname
FROM by_song
WHERE song='All Hands Against His Own'
"""
_query(q)

Row(userid=29, firstname='Jacqueline', lastname='Lynch')
Row(userid=80, firstname='Tegan', lastname='Levine')
Row(userid=95, firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [108]:
session.execute("DROP TABLE by_session;")
session.execute("DROP TABLE by_song;")

### Close the session and cluster connection¶

In [109]:
session.shutdown()
cluster.shutdown()